In [4]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import make_scorer, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [5]:
# Carregar dados
file_path = 'C:\\Users\\JoãoEmileXimenesMuri\\Documents\\pessoal\\TCC\\Bases TCC\\Dados_TCC_2014_2024_Normalizados_TabUnica.xlsx'
data = pd.read_excel(file_path, sheet_name='Sheet1')


In [6]:
# Processar dados
data['Data Referência'] = pd.to_datetime(data['Data Referência'])
data['IPCA Próximo Mês'] = data['Valor IPCA'].shift(-1)
data = data.dropna()

In [7]:
X = data.drop(columns=['Data Referência', 'IPCA Próximo Mês'])
y = data['IPCA Próximo Mês']

In [8]:
# Configurar TimeSeriesSplit
n_splits = 5
tss = TimeSeriesSplit(n_splits=n_splits)


In [9]:
# Definir a métrica como R²
scoring = make_scorer(r2_score)

In [10]:
# Definir os modelos - ADICIONAR REGRESSÃO LINEAR REGULARIZADA
models = {
    "Regressão Linear": LinearRegression(),
    "SVR": SVR(),
    "Decision Tree": DecisionTreeRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "Random Forest": RandomForestRegressor()
}

In [11]:
# Definir grids de parâmetros para cada modelo
param_grids = {
    "SVR": {
        'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'epsilon': [0.001, 0.01, 0.1, 0.5, 1]
    },
    "Decision Tree": {
        'max_depth': [3, 5, 10, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    "Gradient Boosting": {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.001, 0.01, 0.1, 0.5],
        'max_depth': [3, 5, 10],
        'subsample': [0.8, 1.0]
    },
    "Random Forest": {
        'n_estimators': [100, 200, 500],
        'max_depth': [3, 5, 10, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
}

In [12]:
for model_name, model in models.items():
    print(f"Treinando e avaliando o modelo: {model_name}")
    
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grids.get(model_name, {}),
        scoring='r2',
        cv=tss,
        verbose=1,
        n_jobs=-1,
        refit=False  # Mantido como falso para evitar sobrecarga automática
    )
    
    grid_search.fit(X, y)
    
    best_index = grid_search.best_index_
    best_params = grid_search.cv_results_['params'][best_index]
    best_score = grid_search.cv_results_['mean_test_score'][best_index]
    
    print(f"Melhores Parâmetros para o modelo {model_name}: {best_params}")
    print(f"Melhor Pontuação (r2) para o modelo {model_name}: {best_score:.4f}")
    print("=" * 50)
    
    best_model = model.set_params(**best_params)
    best_model.fit(X, y)
    
    y_pred = best_model.predict(X)
    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    
    print(f"Desempenho final para o modelo {model_name}:")
    print(f"  MSE: {mse:.4f}")
    print(f"  R²: {r2:.4f}")
    print("=" * 50)


Treinando e avaliando o modelo: Regressão Linear
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Melhores Parâmetros para o modelo Regressão Linear: {}
Melhor Pontuação (r2) para o modelo Regressão Linear: -3.3076
Desempenho final para o modelo Regressão Linear:
  MSE: 0.0005
  R²: 0.9943
Treinando e avaliando o modelo: SVR
Fitting 5 folds for each of 120 candidates, totalling 600 fits
Melhores Parâmetros para o modelo SVR: {'C': 1, 'epsilon': 0.01, 'kernel': 'linear'}
Melhor Pontuação (r2) para o modelo SVR: -0.7208
Desempenho final para o modelo SVR:
  MSE: 0.0007
  R²: 0.9909
Treinando e avaliando o modelo: Decision Tree
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Melhores Parâmetros para o modelo Decision Tree: {'max_depth': 3, 'min_samples_leaf': 4, 'min_samples_split': 2}
Melhor Pontuação (r2) para o modelo Decision Tree: -1.9856
Desempenho final para o modelo Decision Tree:
  MSE: 0.0029
  R²: 0.9634
Treinando e avaliando o modelo: Gradient Boosting
